# Import the necessary packages

In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, dayofmonth, dayofweek, weekofyear, date_format, from_unixtime

from pyspark.sql.functions import row_number,lit
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, DateType, LongType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1647802554468_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Spark Session
Creating the spark session is not necessary, because it is provided by Amazon EMR.

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark


#spark = create_spark_session()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Process Song data

## Load song data

In [3]:
input_song_data = "s3://udacity-dend/song_data/*/*/*/*.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Schema to read the data from the Json files.

In [4]:
song_data_schema = StructType([
    StructField("artist_id", StringType()),
    StructField("artist_latitude", FloatType()),
    StructField("artist_location", StringType()),
    StructField("artist_longitude", FloatType()),
    StructField("artist_name", StringType()),
    StructField("duration", FloatType()),
    StructField("num_songs", IntegerType()),
    StructField("song_id", StringType()),
    StructField("title", StringType()),
    StructField("year", StringType())
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Load the data

In [5]:
# read song data file
df = spark.read.json(input_song_data,
                     schema=song_data_schema,
                     mode="PERMISSIVE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Verify the schema, the amount of rows loaded and the first 10 rows of the data.

In [6]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: float (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: float (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: float (nullable = true)
 |-- num_songs: integer (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)

In [7]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [8]:
df.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|233.22076|        1|SOVIYJY12AF72A4B00|The Dead Next Doo...|1983|
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|287.92117|        1|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|1983|
|ARQ846I1187B9A7083|           null|                    |            null|Yvonne S. Moriart...|196.04852|        1|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|   0|
|AR4T2IF1187B9ADBB7|       63.9602

## Create song data

From the song data the information for the songs are taken.

In [9]:
# extract columns to create songs table
songs_table = df.select(["song_id",
                         "title",
                         "artist_id",
                         "year",
                         "duration"]).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
songs_table.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOXILZP12A6D4F75C4|Protection (Under...|ARNF13I1187FB562A5|   0|455.70566|
|SOIBFQH12A6D4F8983|Joey (Acoustic) (...|AR56D3Y1187B9AD12A|1994|247.37914|
|SOGRQYN12A8AE47828|Feliz Navidad (W/...|ARB2FS51187FB42679|2005|161.04443|
|SOTDNWF12AB0186452|   Revolution Action|AR3TG3X1187FB4D577|1999|249.46893|
|SOMTMKQ12A6D4FA3A7|This Is The Dream...|AR0687M1187FB5604E|2005|204.35545|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows

There should be only one row for each song.

In [11]:
songs_table.count(), songs_table.select("song_id").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(14896, 14896)

Save data

In [12]:
(
    songs_table
    .write.format("parquet")
    .mode('overwrite')
    .partitionBy('year', 'artist_id')
    .save('s3://datalakepractice032022/songs')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create artist data
From the song data the information for the artist are taken.

First the information is group by the artist name and then by artist id to make sure that the names and the ids are unique.

In [13]:
# extract columns to create songs table
artists_table =(
    df.select(["artist_id", "artist_name", "artist_location", "artist_latitude","artist_longitude"])
    .groupBy('artist_name')
    .agg(F.max('artist_id'), F.max('artist_location'), F.max('artist_latitude'), F.max('artist_longitude'))
    .withColumnRenamed("max(artist_id)", "id")
    .withColumnRenamed("artist_name", "name")
    .withColumnRenamed("max(artist_location)", "location")
    .withColumnRenamed("max(artist_latitude)","lattitude")
    .withColumnRenamed("max(artist_longitude)", "longitude")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
artists_table_renamed = (
    artists_table
    .groupBy('id')
    .agg(F.max('name'), F.max('location'), F.max('lattitude'), F.max('longitude'))
    .withColumnRenamed("max(name)", "name")
    .withColumnRenamed("max(location)", "location")
    .withColumnRenamed("max(lattitude)","lattitude")
    .withColumnRenamed("max(longitude)", "longitude")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
artists_table_renamed.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- lattitude: float (nullable = true)
 |-- longitude: float (nullable = true)

There should be only one row for each artist.

In [16]:
artists_table_renamed.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+--------------------+---------+---------+
|                id|             name|            location|lattitude|longitude|
+------------------+-----------------+--------------------+---------+---------+
|AR0G85S1187FB4D46D|    Hey Hey My My|                    |     null|     null|
|AR13CXU1187B9AD30A|        The Clash|     London, England| 51.52328| -0.21346|
|AR1X3SU11C8A415261|          Vessels|Hanover Park/Scha...| 41.99112|-88.15849|
|AR23GZ01187B98BC19|Whatever It Takes|                    |     null|     null|
|AR29ZZM1187B9B04C7|      John Powell|                    |     null|     null|
+------------------+-----------------+--------------------+---------+---------+
only showing top 5 rows

In [17]:
print(artists_table_renamed.count(),
      artists_table_renamed.select("id").distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9511 9511

The information is saved back to S3

In [18]:
(
    artists_table_renamed
    .write.format("parquet")
    .mode('overwrite')
    .save('s3://datalakepractice032022/artists')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Process Log data

## Load the data

In [19]:
input_log_data = "s3://udacity-dend/log_data/*/*/*.json"
input_log_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://udacity-dend/log_data/*/*/*.json'

Schema to read the data from the Json files.

In [20]:
log_data_schema = StructType([
    StructField("artist", StringType()),
    StructField("auth", StringType()),
    StructField("firstName", StringType()),
    StructField("gender", StringType()),
    StructField("itemInSession", StringType()),
    StructField("lastName", StringType()),
    StructField("length", FloatType()),
    StructField("level", StringType()),
    StructField("location", StringType()),
    StructField("method", StringType()),
    StructField("page", StringType()),
    StructField("registration", StringType()),
    StructField("sessionId", IntegerType()),
    StructField("song", StringType()),
    StructField("status", IntegerType()),
    StructField("ts", LongType()),
    StructField("userAgent", StringType()),
    StructField("userId", StringType())
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Load the data and cast the Unix Epoch time to a timestamp column.

In [21]:
# read song data file
spark.conf.set("spark.sql.session.timeZone", "America/Los_Angeles")
df_log = spark.read.json(input_log_data,
                     schema=log_data_schema,
                     mode="PERMISSIVE").withColumn('start_time',from_unixtime(col("ts")/1000))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Verify the conversion of the dates was done correctly

In [22]:
df_log.select(['ts', 'start_time']).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------------+
|           ts|         start_time|
+-------------+-------------------+
|1542241826796|2018-11-14 16:30:26|
|1542242481796|2018-11-14 16:41:21|
|1542242741796|2018-11-14 16:45:41|
|1542247071796|2018-11-14 17:57:51|
|1542252577796|2018-11-14 19:29:37|
|1542253449796|2018-11-14 19:44:09|
|1542253460796|2018-11-14 19:44:20|
|1542260074796|2018-11-14 21:34:34|
|1542260277796|2018-11-14 21:37:57|
|1542260935796|2018-11-14 21:48:55|
+-------------+-------------------+
only showing top 10 rows

Verify the schema, the amount of rows loaded and the first 10 rows of the data.

In [23]:
df_log.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: float (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: string (nullable = true)
 |-- sessionId: integer (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: string (nullable = true)

In [24]:
df_log.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

In [25]:
df_log.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|     artist|      auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|         start_time|
+-----------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|   Harmonia| Logged In|     Ryan|     M|            0|   Smith| 655.7775| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-14 16:30:26|
|The Prodigy| Logged In|     Ryan|     M|            1| 

## Create songplay data

In [26]:
df_log.select(["page"]).distinct().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(page='Submit Downgrade'), Row(page='Home'), Row(page='Downgrade'), Row(page='Logout'), Row(page='Save Settings'), Row(page='About'), Row(page='Settings'), Row(page='Login'), Row(page='NextSong'), Row(page='Help'), Row(page='Upgrade'), Row(page='Error'), Row(page='Submit Upgrade')]

We are interested only in song reproductions, so we filter when page is equal to *NextSong*.

After the data is filtered we need to get to add the columns: *song_id*, *artist_id*. So we joined it with: *artists_table* and *song_table* 

In [27]:
artists_table_join = artists_table_renamed.select(['id', 'name'])
song_table_join = songs_table.select(["song_id", "title", "artist_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_log_filtered = df_log.where(df_log.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_log_joined = (
    df_log_filtered
    .join(artists_table_join,
          df_log_filtered.artist == artists_table_join.name,
          how='left_outer')
)

df_log_joined = (
    df_log_joined
    .join(song_table_join,
          (song_table_join.artist_id == df_log_joined.id) & (song_table_join.title == df_log_joined.song),
          how='left_outer'
         )
)
df_log_joined.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: float (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: string (nullable = true)
 |-- sessionId: integer (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)

In [30]:
df_log_joined.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+------------------+------------+-------+-----+---------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|         start_time|                id|        name|song_id|title|artist_id|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+------------------+------------+-------+-----+---------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith| 655.7775| free|San Jose-Sun

Verify that before and after the joined the amount of rows are the same. It means that the data was not duplicated.

In [31]:
print(df_log_filtered.count(),
      df_log_joined.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6820 6820

Songplay id is created.

In [32]:
w = Window().orderBy(lit('A'))

df_id = df_log_joined.withColumn("songplay_id", row_number().over(w))
df_id.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+------------------+-------------+-------+-----+---------+-----------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|         start_time|                id|         name|song_id|title|artist_id|songplay_id|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+------------------+-------------+-------+-----+---------+-----------+
|       Stephen Lynch|Logged In|   Jayden|     M|            

The columns year and month are created so the data can be partioned. 

In [33]:
songplays_table = (
    df_id
    .withColumn('year', year(col('start_time')))
    .withColumn('month', month(col('start_time')))
    .select([
    'songplay_id',
    'start_time', 
    'userId',
    'level',
    'song',
    'artist',
    'sessionId',
    'location',
    'userAgent',
    'year',
    'month'
    ])
)
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- songplay_id: integer (nullable = true)
 |-- start_time: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- sessionId: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

In [34]:
songplays_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+------+-----+--------------------+--------------------+---------+--------------------+--------------------+----+-----+
|songplay_id|         start_time|userId|level|                song|              artist|sessionId|            location|           userAgent|year|month|
+-----------+-------------------+------+-----+--------------------+--------------------+---------+--------------------+--------------------+----+-----+
|          1|2018-11-29 16:22:07|    91| free|   Jim Henson's Dead|       Stephen Lynch|      829|Dallas-Fort Worth...|Mozilla/5.0 (comp...|2018|   11|
|          2|2018-11-29 17:08:41|    73| paid|         Shell Shock|             Manowar|     1049|Tampa-St. Petersb...|"Mozilla/5.0 (Mac...|2018|   11|
|          3|2018-11-29 17:12:48|    73| paid|Women Lose Weight...|           Morcheeba|     1049|Tampa-St. Petersb...|"Mozilla/5.0 (Mac...|2018|   11|
|          4|2018-11-29 17:17:05|    73| paid|Won't Go Home Wit...|            Maroon 5|

Save the data

In [35]:
(
    songplays_table
    .write.format("parquet")
    .partitionBy('year', 'month')
    .mode('overwrite')
    .save('s3://datalakepractice032022/songplays')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create user

The most recent information of each user is taken and saved.

In [36]:
sw2 = Window.partitionBy("userId").orderBy(col("start_time").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
users_table =(
    df_log
    .withColumn("row", row_number().over(sw2))
    .filter(col("row") == 1)
    .drop("row")
    .select(["userId", "firstName", "lastName", "gender","level"])
    .withColumnRenamed("userId", "user_id")
    .withColumnRenamed("firstName", "first_name")
    .withColumnRenamed("lastName", "last_name")
)
users_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [38]:
(
    users_table
    .write.format("parquet")
    .mode('overwrite')
    .save('s3://datalakepractice032022/users')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create time

The unique start times are taken and the addional information is calculated and then saved.

In [39]:
time_df = df_log.select('start_time').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
time_df_final = (
    time_df
    .withColumn('hour', hour(col('start_time')))
    .withColumn('day', dayofmonth(col('start_time')))
    .withColumn('week', weekofyear(col('start_time')))
    .withColumn('month', month(col('start_time')))     
    .withColumn('year', year(col('start_time')))
    .withColumn('weekday', dayofweek(col('start_time')))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
time_df_final.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-28 15:56:53|  15| 28|  48|   11|2018|      4|
|2018-11-05 07:43:22|   7|  5|  45|   11|2018|      2|
|2018-11-05 08:53:09|   8|  5|  45|   11|2018|      2|
|2018-11-12 20:49:34|  20| 12|  46|   11|2018|      2|
|2018-11-26 19:53:03|  19| 26|  48|   11|2018|      2|
|2018-11-23 07:29:23|   7| 23|  47|   11|2018|      6|
|2018-11-23 08:05:27|   8| 23|  47|   11|2018|      6|
|2018-11-23 08:44:59|   8| 23|  47|   11|2018|      6|
|2018-11-23 13:25:15|  13| 23|  47|   11|2018|      6|
|2018-11-18 11:29:36|  11| 18|  46|   11|2018|      1|
+-------------------+----+---+----+-----+----+-------+
only showing top 10 rows

In [42]:
(
    time_df_final
    .write.format("parquet")
    .partitionBy('year', 'month')
    .mode('overwrite')
    .save('s3://datalakepractice032022/time')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…